## Integrating SQLModel, Datasette (SQLite_Utils) and Holoviz Panel

This is a proof-of-concept / draft repository.

My name for this project is 'BeeDiscovery': a shortform for 'Better eDiscovery', primarily motivated by the inefficiencies I have found when attempting to validate tabular databases which we have recieved from other parties, and as a standardizing tool for my forensic and ediscovery team to massage and impose some semblance of order on the various datatypes that we encounter during all stages of cybersecurity incident response and our expert testimony work.

End Goal: ideally we can easily and repeatably dump data sources into sqlite (either through our own loaders or by leveraging existing loader interfaces which can dump to pandas / sqlite) and then triage the columns, assign roles and kick-off downstream processes.


__Future ideas:__

 - have a function which can generate datasette metadata .json objects: https://docs.datasette.io/en/stable/metadata.html
 - functionality to create / dump / restore ArangoDB collections to SQLite for long-term persistence.



__Models__

 - *Dataset*
    - As many as you want in a single sqlite database
    - link to a `sqlite_utils` Table object for a table named for the `db_name` attribute
    - Future: if ArangoDB is enabled in the environment, also set up connection to a data collection and graph when the attributes are first called.
 - *DataField*
    - attached to a Dataset, describes metadata about a column. Generally created by reading data from the source table, using `Dataset.sync_columns()`
 - *DataRole*
    - describes how the contents of a column can be used. More of a tag on a column, eventually there will be validators and other checks.
    - processes should ideally be written referring to DataRoles, not DataFields, since the fields can change from Dataset-to-Dataset, but Roles are eternal.
 - *DataFieldRoleLink*: m2m link with a priority attribute (WIP)
    - just a class used by SQLModel/SQLAlchemy for the m2m relationship.
    - also has a `priority` attribute, but not used. Eventually may be used for sorting.
 - *BeeDiscovery*: general settings class, generator for new Datasets, stores the sqlalchemy `._session`


__Helper Classes__

I have done some experimentation with in the helpers.py module, in `DynamicAttrDefaultDictList`, which takes in a list or dict and supports IDE autocompletion generated from attribute values. This is pretty convenient when working on flexible data tables, where you aren't always certain what columns are there, or don't want to type them in all the time in `['item']` notation.

This is also nice be because, once a DataRole has been assigned to a DataField (for example, the DOCDATE), you can write repeatable processes which refer to the `dataset.r.DOCDATE` and it will always know what you're referring to. You can thus write simple scripts / validations without needing to always map back to the specific columns in the target datasets.

__SQLite Tables:__

in addition to these metadata tables, for each dataset we also store:

- dataset_[whatever the Dataset internal name is] - data table for the actual content




In [1]:
# DynamicAttrDefaultDictList works with jedi, you might need to turn it on
# %config IPCompleter.use_jedi = True

## Getting Started

We will first create a BeeDiscovery object, save a sqlite database and create some example DataRoles.

In [2]:
from sqlmodels import DataRole, DataField, DataFieldRoleLink, Dataset, BeeDiscovery

In [3]:
bee = BeeDiscovery.load('bee.beedb')

In [4]:
bee.test_option='hello there, this is a test'

now we can manually commit. I know that this should probably be moved into a context manager for each function.
It's still PoC.

In [5]:
bee._session.commit()

using sqlite_utils, let's check the raw contents of the sqlalchemy tables:

In [6]:
bee.db.tables

[<Table __beediscovery (id, name, beed_file_path, test_option)>,
 <Table __beed_datarole (id, name, is_unique, beediscovery_id)>,
 <Table __beed_dataset (id, name, table, beediscovery_id)>,
 <Table __beed_datafield (id, db_name, db_type, db_default_value, db_is_primary_key, name, is_json, priority, description, dataset_id)>,
 <Table __beed_datafieldrolelink (field_id, role_id, priority)>,
 <Table students (id, name, class, mark, gender)>]

In [7]:
[x for x in bee.db['__beed_dataset'].rows]

[{'id': 1, 'name': 'students', 'table': 'students', 'beediscovery_id': 1}]

check for persistence by recreating from the database:

In [8]:
del bee

In [9]:
bee = BeeDiscovery.load('bee.beedb')
bee.test_option

'hello there, this is a test'

Next we're going to do some tests with Panel UI elements generated from the Pydantic attributes helpfully incorporated into SQLModel.

In [10]:
from sqlmodel import create_engine, SQLModel, Session, select
from sqlite_utils import Database

# first get panel setup for jupyter.
import panel as pn
pn.extension('gridstack')

In [11]:
bee.datasets

[Dataset: students, 2800 records with DataFields ['id', 'name', 'class', 'mark', 'gender']]

we're fixing the ids here, there is a validator which swaps out transient sqlalchemy items for records which are already in the database with these session ids.



In [12]:
BEGDOC = DataRole(name='BEGDOC', is_unique=True, id=1)
BEGATT = DataRole(name='BEGATT', id=2)
DOCDATE = DataRole(name='DOCDATE', id=3)

### Load some data
Fetch some test data. We will use https://www.plus2net.com/python/download/student.csv

In [13]:
import pandas as pd
#bee['students'].t.insert_all(pd.read_csv('https://www.plus2net.com/python/download/student.csv').to_dict(orient='records'))

now that the data is loaded into the sqlite backing table, lets generate some DataFields for the columns:

In [14]:
students = bee['students']
students.sync_columns()

{'matched_datafields': [DataField: id, in dataset students with role(s): ['BEGATT'],
  DataField: name, in dataset students,
  DataField: class, in dataset students,
  DataField: mark, in dataset students,
  DataField: gender, in dataset students with role(s): ['BEGATT', 'DOCDATE']],
 'extra_datafields': []}

now we have datafields, one for each column which we loaded from the CSV.
We can run `.first_nonblank()` on each one to sample the actual column values.

In [15]:
for field in students.fields:
    field: DataField
    print(field.name, field.first_nonblank())

id [1]
name ['John Deo']
class ['Four']
mark [75]
gender ['female']


### Create some DataRoles

We have to get these loaded. Ideally these will be loaded into a DataRole registry from a plugin system.

In [16]:
for role in [BEGDOC, BEGATT, DOCDATE]:
    
    if role not in bee.roles:
        # print("it isn't there")
        bee.roles.append(role)

aaaand save.

In [17]:
bee._session.commit()

once again, let's inspect the raw backend table to see our new DataFields using our sqlite_utils connection:

In [18]:
[x for x in bee.db['__beed_datafield'].rows]

[{'id': 1,
  'db_name': 'id',
  'db_type': 'INTEGER',
  'db_default_value': None,
  'db_is_primary_key': 0,
  'name': 'id',
  'is_json': 0,
  'priority': None,
  'description': None,
  'dataset_id': 1},
 {'id': 2,
  'db_name': 'name',
  'db_type': 'TEXT',
  'db_default_value': None,
  'db_is_primary_key': 0,
  'name': 'name',
  'is_json': 0,
  'priority': None,
  'description': None,
  'dataset_id': 1},
 {'id': 3,
  'db_name': 'class',
  'db_type': 'TEXT',
  'db_default_value': None,
  'db_is_primary_key': 0,
  'name': 'class',
  'is_json': 0,
  'priority': None,
  'description': None,
  'dataset_id': 1},
 {'id': 4,
  'db_name': 'mark',
  'db_type': 'INTEGER',
  'db_default_value': None,
  'db_is_primary_key': 0,
  'name': 'mark',
  'is_json': 0,
  'priority': None,
  'description': None,
  'dataset_id': 1},
 {'id': 5,
  'db_name': 'gender',
  'db_type': 'TEXT',
  'db_default_value': None,
  'db_is_primary_key': 0,
  'name': 'gender',
  'is_json': 0,
  'priority': None,
  'description'

### Assigning Data
Now that we have data, DataFields and DataRoles, we can assign DataRoles to DataFields.

We will use the `bee.d.students.ra` property method to fetch the DataRole object, which allows you to autocomplete DataRoles which are available for that Dataset.

Assigning a role to a DataField:

In [19]:
bee.d.students.f.gender.roles.options

[DataRole: DOCDATE, used by DataFields ['gender'],
 DataRole: BEGATT, used by DataFields ['id', 'gender']]

In [20]:
bee.d.students.f.gender.roles.append(bee.d.students.ra.DOCDATE)

validating the 'role', changed to ('roles', DataRole: DOCDATE, used by DataFields ['gender'], <class 'sqlmodels.DataRole'>)


check that the role was assigned correctly:

In [21]:
bee.d.students.fields

[DataField: id, in dataset students with role(s): ['BEGATT'],
 DataField: name, in dataset students,
 DataField: class, in dataset students,
 DataField: mark, in dataset students,
 DataField: gender, in dataset students with role(s): ['BEGATT', 'DOCDATE', 'DOCDATE']]

## Testing `pydantic-panel`

In [22]:
# first get panel setup for jupyter.
import panel as pn
pn.extension('gridstack')
datafield_gender = bee.d.students.f.gender

### Testing with 'out-of-the-box' functionality

By design, `pydantic-python` will recurse through attributes in a pydantic model and assemble a composite widget from their contents. As such, by default all pydantic fields (members of the `__fields__` attribute) will be included. This works perfectly:

In [23]:
datafield_gender.roles

[DataRole: BEGATT, used by DataFields ['id', 'gender'],
 DataRole: DOCDATE, used by DataFields ['gender', 'gender'],
 DataRole: DOCDATE, used by DataFields ['gender', 'gender']]

In [24]:
pn.panel(datafield_gender)

PydanticModelEditor(_widgets={'id': IntInput(name='id',...}, class_=<class 'sqlmodels.DataFiel..., value=DataField: gender, ...)

The challenge now is that we don't all of the fields to be visible, or to be editable. We want more control over what is shown above.

### Testing with a custom Editor

we have a custom editor and dispatcher within our `pydantic_panel_widgets` module. after importing it, our custom editor will be used to render DataField objects.

In [25]:
from pydantic_panel_widgets import *

now we will run it again, and the dispatcher will use our custom editor:

In [26]:
type(datafield_gender.roles)

helpers.OptionedList

In [27]:
datafield_gender.roles

[DataRole: BEGATT, used by DataFields ['id', 'gender'],
 DataRole: DOCDATE, used by DataFields ['gender', 'gender'],
 DataRole: DOCDATE, used by DataFields ['gender', 'gender']]

In [28]:
pn.panel(datafield_gender.roles)

using infer_widget for OptionedList: type(value)=<class 'helpers.OptionedList'> and value=[DataRole: BEGATT, used by DataFields ['id', 'gender'], DataRole: DOCDATE, used by DataFields ['gender', 'gender'], DataRole: DOCDATE, used by DataFields ['gender', 'gender']]


value=DataField: gender, in dataset students with role(s): ['DOCDATE'] 
value=DataField: gender, in dataset students with role(s): ['DOCDATE'] 
validating the 'role', changed to ('roles', DataRole: BEGATT, used by DataFields ['id'], )

MultiChoice(options={'DOCDATE': DataRole: DOCD...}, solid=False, value=[DataRole: BEGATT, ...])

In [42]:
pn.panel(datafield_gender.roles)

using infer_widget for OptionedList: type(value)=<class 'helpers.OptionedList'> and value=[DataRole: DOCDATE, used by DataFields ['gender'], DataRole: BEGATT, used by DataFields ['id', 'gender']]


value=DataField: gender, in dataset students with role(s): ['DOCDATE', 'BEGATT']

MultiChoice(options={'DOCDATE': DataRole: DOCD...}, solid=False, value=[DataRole: DOCDATE, ...])

now, since we have modified the `__init__` method of the Editor used above, we can see the ROLES multichoice field, which has a dynamic watcher to update the widget's available options whenever the widget value is changed.

In [30]:
datafield_gender.roles._sa_adapter._key

'roles'

In [31]:
widget = pn.panel(datafield_gender.roles, bidirectional=True)

using infer_widget for OptionedList: type(value)=<class 'helpers.OptionedList'> and value=[DataRole: BEGATT, used by DataFields ['id', 'gender'], DataRole: DOCDATE, used by DataFields ['gender', 'gender'], DataRole: DOCDATE, used by DataFields ['gender', 'gender']]


In [45]:
print(widget.value)

[DataRole: BEGATT, used by DataFields ['id', 'gender']]


In [46]:
x = pn.widgets.IntSlider(name='x', start=0, end=100)

In [34]:
bee._session.commit()

In [35]:
students.r

{'BEGATT': [DataField: id, in dataset students with role(s): ['BEGATT'], DataField: gender, in dataset students with role(s): ['BEGATT', 'DOCDATE']], 'DOCDATE': DataField: gender, in dataset students with role(s): ['BEGATT', 'DOCDATE']}

In [36]:
datafield_gender.roles

[DataRole: BEGATT, used by DataFields ['id', 'gender'],
 DataRole: DOCDATE, used by DataFields ['gender']]

In [37]:
print(type(datafield_gender.roles))
from helpers import OptionedList
isinstance(datafield_gender.roles, list)

<class 'helpers.OptionedList'>


True

In [38]:
isinstance(datafield_gender.roles, list)

True

In [39]:
from plum import type_of

In [40]:
type_of(datafield_gender.roles)

List[sqlmodels.DataRole]

In [41]:
for x in datafield_gender.roles:
    if x in datafield_gender.roles_available:
        print(x)

is_unique=False name='BEGATT' id=2 beediscovery_id=1 fields=[DataField: id, in dataset students with role(s): ['BEGATT'], DataField: gender, in dataset students with role(s): ['BEGATT', 'DOCDATE']]
is_unique=False name='DOCDATE' id=3 beediscovery_id=1 fields=[DataField: gender, in dataset students with role(s): ['BEGATT', 'DOCDATE']]
